In [1]:

import tensorflow as tf
print(tf.__version__)


2.16.1


In [2]:
pip install --upgrade tensorflow

Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install --upgrade tensorflow


Note: you may need to restart the kernel to use updated packages.


In [4]:
import tensorflow as tf
print(tf.__version__)


2.16.1


In [5]:
pip install keras


Note: you may need to restart the kernel to use updated packages.


In [6]:
from keras.layers import Input


In [7]:
from tensorflow.keras.layers import Input
import tensorflow as tf
import pathlib
from tensorflow.keras.initializers import RandomNormal
from tensorflow.keras.layers import Input, Conv2D, Conv2DTranspose, LeakyReLU, Activation, Concatenate, Dropout, BatchNormalization
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Model
from numpy import zeros, ones
from numpy.random import randint
from matplotlib import pyplot as plt


In [8]:
import numpy as np
import os
from skimage.transform import resize
from matplotlib import pyplot as plt

# Define file paths
input_dir = r"C:\Users\net pc\Desktop\LIVER TUMOR\benign_input"
label_dir = r"C:\Users\net pc\Desktop\LIVER TUMOR\benign_output"

def load_data(input_dir, label_dir, target_shape):
    input_files = os.listdir(input_dir)
    label_files = os.listdir(label_dir)
    
    # Sort files to ensure corresponding inputs and labels match
    input_files.sort()
    label_files.sort()
    
    trainA = []
    trainB = []
    for i in range(len(input_files)):
        input_img = plt.imread(os.path.join(input_dir, input_files[i]))
        label_img = plt.imread(os.path.join(label_dir, label_files[i]))
        
        # Resize images to target shape if they have different shapes
        if input_img.shape != target_shape:
            input_img = resize(input_img, target_shape)
        if label_img.shape != target_shape:
            label_img = resize(label_img, target_shape)
        
        trainA.append(input_img)
        trainB.append(label_img)
    
    return np.array(trainA), np.array(trainB)



# Define target shape for images (e.g., (256, 256, 3))
target_shape = (256, 256, 3)

# Load data
data = load_data(input_dir, label_dir, target_shape)
print("Data loaded successfully!")



Data loaded successfully!


In [9]:
import tensorflow as tf
from tensorflow.keras.layers import Layer

class InstanceNormalization(Layer):
    def __init__(self, epsilon=1e-5):
        super(InstanceNormalization, self).__init__()
        self.epsilon = epsilon

    def build(self, input_shape):
        self.scale = self.add_weight(name='scale',
                                     shape=input_shape[-1:],
                                     initializer=tf.random_normal_initializer(1., 0.02),
                                     trainable=True)
        self.offset = self.add_weight(name='offset',
                                      shape=input_shape[-1:],
                                      initializer='zeros',
                                      trainable=True)

    def call(self, inputs):
        mean, variance = tf.nn.moments(inputs, axes=[1, 2], keepdims=True)
        inv = tf.math.rsqrt(variance + self.epsilon)
        normalized = (inputs - mean) * inv
        return self.scale * normalized + self.offset


In [10]:
from keras.layers import Conv2D, Input, Concatenate, LeakyReLU, Activation
from keras.initializers import RandomNormal
from keras.models import Model

def define_discriminator(image_shape):
    init = RandomNormal(stddev=0.02)
    in_src_image = Input(shape=image_shape, name='input_image')
    in_target_image = Input(shape=image_shape, name='target_image')
    merged = Concatenate()([in_src_image, in_target_image])
    
    # Discriminator architecture
    d = Conv2D(64, (4,4), strides=(2,2), padding='same', kernel_initializer=init)(merged)
    d = LeakyReLU(alpha=0.2)(d)
    
    d = Conv2D(128, (4,4), strides=(2,2), padding='same', kernel_initializer=init)(d)
    d = InstanceNormalization()(d)
    d = LeakyReLU(alpha=0.2)(d)
    
    d = Conv2D(256, (4,4), strides=(2,2), padding='same', kernel_initializer=init)(d)
    d = InstanceNormalization()(d)
    d = LeakyReLU(alpha=0.2)(d)
    
    d = Conv2D(512, (4,4), strides=(2,2), padding='same', kernel_initializer=init)(d)
    d = InstanceNormalization()(d)
    d = LeakyReLU(alpha=0.2)(d)
    
    d = Conv2D(512, (4,4), padding='same', kernel_initializer=init)(d)
    d = InstanceNormalization()(d)
    d = LeakyReLU(alpha=0.2)(d)
    
    patch_out = Conv2D(1, (4,4), padding='same', kernel_initializer=init)(d)
    patch_out = Activation('sigmoid')(patch_out)
    
    model = Model([in_src_image, in_target_image], patch_out)
    model.compile(loss='binary_crossentropy', optimizer='adam')
   
    return model


In [11]:
from tensorflow.keras.layers import PReLU

def define_generator(image_shape=(256, 256, 3)):
    init = RandomNormal(stddev=0.02)
    in_image = Input(shape=image_shape)
    
    # Encoder
    e1 = Conv2D(64, (4, 4), strides=(2, 2), padding='same', kernel_initializer=init)(in_image)
    e1 = LeakyReLU(alpha=0.2)(e1)
    
    e2 = Conv2D(128, (4, 4), strides=(2, 2), padding='same', kernel_initializer=init)(e1)
    e2 = BatchNormalization()(e2)
    e2 = LeakyReLU(alpha=0.2)(e2)
    
    e3 = Conv2D(256, (4, 4), strides=(2, 2), padding='same', kernel_initializer=init)(e2)
    e3 = BatchNormalization()(e3)
    e3 = LeakyReLU(alpha=0.2)(e3)
    
    # Bottleneck
    b = Conv2D(512, (4, 4), strides=(2, 2), padding='same', kernel_initializer=init)(e3)
    b = PReLU(shared_axes=[1, 2])(b)  # Use PReLU from tf.keras.layers
    
    # Decoder
    d1 = Conv2DTranspose(512, (4, 4), strides=(2, 2), padding='same', kernel_initializer=init)(b)
    d1 = BatchNormalization()(d1)
    d1 = Activation('relu')(d1)
    d1 = Concatenate()([d1, e3])
    
    d2 = Conv2DTranspose(256, (4, 4), strides=(2, 2), padding='same', kernel_initializer=init)(d1)
    d2 = BatchNormalization()(d2)
    d2 = Activation('relu')(d2)
    d2 = Concatenate()([d2, e2])
    
    d3 = Conv2DTranspose(128, (4, 4), strides=(2, 2), padding='same', kernel_initializer=init)(d2)
    d3 = BatchNormalization()(d3)
    d3 = Activation('relu')(d3)
    d3 = Concatenate()([d3, e1])
    
    # Output
    g = Conv2DTranspose(image_shape[2], (4, 4), strides=(2, 2), padding='same', kernel_initializer=init)(d3)
    out_image = Activation('tanh')(g)
    
    model = Model(in_image, out_image)
    return model


In [12]:
def generate_real_samples(dataset, n_samples, patch_shape):
    # unpack dataset
    trainA, trainB = dataset
    # choose random instances
    ix = randint(0, trainA.shape[0], n_samples)
    # retrieve selected images
    X1, X2 = trainA[ix], trainB[ix]
    # generate 'real' class labels (1)
    y = ones((n_samples, 1))
    return [X1, X2], y


In [13]:
# generate a batch of images, returns images and targets
def generate_fake_samples(g_model, samples, patch_shape):
	# generate fake instance
	X = g_model.predict(samples)
	# create 'fake' class labels (0)
	y = zeros((len(X), patch_shape, patch_shape, 1))
	return X, y


In [14]:
def define_gan(g_model, d_model, image_shape):
    # Make weights in the discriminator not trainable
    d_model.trainable = False
    
    # Input tensor for the generator model
    gen_input = Input(shape=image_shape)
    
    # Connect the generator output to the discriminator input
    gan_output = d_model([gen_input, g_model(gen_input)])
    
    # Define GAN model taking generator input and outputting discriminator decision
    gan_model = Model(gen_input, gan_output)
    
    # Compile GAN model
    gan_model.compile(loss='binary_crossentropy', optimizer=Adam(learning_rate=0.0002, beta_1=0.5))
    
    return gan_model

# Define discriminator model
d_model = define_discriminator(target_shape)

# Define generator model
g_model = define_generator(target_shape)

# Define GAN model
gan_model = define_gan(g_model, d_model, target_shape)


C:\Users\Public\Efendim\Lib\site-packages\keras\src\layers\activations\leaky_relu.py:41: UserWarning: Argument `alpha` is deprecated. Use `negative_slope` instead.
  warnings.warn(


In [15]:

import numpy as np

def train(d_model, g_model, gan_model, dataset, n_epochs=1000, n_batch=1):
    # Determine the output square shape of the discriminator
    n_patch = d_model.output_shape[1]
    # Unpack dataset
    trainA, trainB = dataset
    # Calculate the number of batches per training epoch
    bat_per_epo = int(len(trainA) / n_batch)  # Convert to integer
    # Calculate the total number of training iterations
    n_steps = bat_per_epo * n_epochs
    
    for i in range(n_steps):
        # Select a batch of real samples
        [X_realA, X_realB], y_real = generate_real_samples(dataset, n_batch, n_patch)
        # Generate a batch of fake samples
        X_fakeB, _ = generate_fake_samples(g_model, X_realA, n_patch)
        # Update discriminator for real samples
        d_loss1 = d_model.train_on_batch([X_realA, X_realB], np.ones((n_batch, n_patch, n_patch, 1)))
        # Update discriminator for fake samples
         # Update discriminator for generated samples
        d_loss2 = d_model.train_on_batch([X_realA, X_fakeB], np.zeros((n_batch, n_patch, n_patch, 1)))
        # Update the generator
        g_loss = gan_model.train_on_batch(X_realA, [np.ones((n_batch, n_patch, n_patch, 1)), X_realB])
        # Summarize performance
        print('>%d, d1[%.3f] d2[%.3f] g[%.3f]' % (i+1, d_loss1, d_loss2, g_loss))
        # Summarize model performance
        if (i+1) % (bat_per_epo * 10) == 0:
            summarize_performance(i, g_model, dataset)



In [16]:

# generate samples and save as a plot and save the model
#GAN models do not converge, we just want to find a good balance between
#the generator and the discriminator. Therefore, it makes sense to periodically
#save the generator model and check how good the generated image looks. 
import os
from PIL import Image

from PIL import Image


def save_img(filename, image):
    """Save the image to the specified filename."""
    # Scale the image back to the range [0, 255] before saving
    scaled_image = (image * 255).astype(np.uint8)
    # Convert to PIL Image
    im = Image.fromarray(scaled_image)
    # Save the image
    im.save(filename)
    
# Summarize performance
def summarize_performance(step, g_model, dataset, n_samples=3):
    print("Saving images...")
    # Select a sample of input images
    [X_realA, X_realB], _ = generate_real_samples(dataset, n_samples, 1)
    # Generate a batch of fake samples
    X_fakeB, _ = generate_fake_samples(g_model, X_realA, 1)
    # Scale all pixels from [-1,1] to [0,1]
    X_realA = (X_realA + 1) / 2.0
    X_realB = (X_realB + 1) / 2.0
    X_fakeB = (X_fakeB + 1) / 2.0
    

    output_path = "C:\\Users\\net pc\\Desktop\\LIVER TUMOR\\benign_generated"


    for i in range(n_samples):
        # Save real source image as JPEG
        real_source_image = X_realA[i]
        real_source_filename = os.path.join(output_path, f'real_source_{step}_{i + 1}.jpeg')
        save_img(real_source_filename, real_source_image)

        # Save generated target image as JPEG
        generated_target_image = X_fakeB[i]
        generated_target_filename = os.path.join(output_path, f'generated_target_{step}_{i + 1}.jpeg')
        save_img(generated_target_filename, generated_target_image)

        # Save real target image as JPEG
        real_target_image = X_realB[i]
        real_target_filename = os.path.join(output_path, f'real_target_{step}_{i + 1}.jpeg')
        save_img(real_target_filename, real_target_image)

    # Save the generator model
    model_filename = os.path.join(output_path, f'model_{step}.h5')
    g_model.save(model_filename)

    print(f"Images and model saved at {output_path}")

    


In [ ]:
# Train Pix2Pix models using the GAN model
train(d_model, g_model, gan_model, data, n_epochs=1000, n_batch=1)



1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 159ms/step


C:\Users\Public\Efendim\Lib\site-packages\keras\src\backend\tensorflow\trainer.py:74: UserWarning: The model does not have any trainable weights.
  warnings.warn("The model does not have any trainable weights.")


>1, d1[0.603] d2[0.870] g[0.584]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
>2, d1[0.785] d2[0.880] g[0.591]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step
>3, d1[0.827] d2[0.880] g[0.543]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step
>4, d1[0.846] d2[0.880] g[0.510]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step
>5, d1[0.850] d2[0.881] g[0.474]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
>6, d1[0.855] d2[0.880] g[0.436]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
>7, d1[0.860] d2[0.881] g[0.401]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
>8, d1[0.862] d2[0.879] g[0.376]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step
>9, d1[0.864] d2[0.880] g[0.350]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
>10, d1[0.866] d2[0.881] g[0.330]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
>11, d1[0.868] d2[0.881] g[0.311]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
>12, d1[0.868] d2[0.880] g[0.295]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
>13, d1[0.869] d2[0.880] g[0.280]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
>14, d1[0.870] d2[0.881] g[0.267]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
>

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step
>65, d1[0.881] d2[0.883] g[0.085]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step
>66, d1[0.881] d2[0.883] g[0.084]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step
>67, d1[0.881] d2[0.883] g[0.083]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step
>68, d1[0.881] d2[0.883] g[0.082]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step
>69, d1[0.881] d2[0.884] g[0.082]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step
>70, d1[0.882] d2[0.884] g[0.081]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step
>71, d1[0.882] d2[0.884] g[0.080]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step
>72, d1[0.882] d2[0.884] g[0.079]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 51ms/step
>73, d1[0.882] d2[0.884] g[0.078]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step
>74, d1[0.883] d2[0.884] g[0.077]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step
>75, d1[0.882] d2[0.884] g[0.076]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step
>76, d1[0.883] d2[0.885] g[0.075]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step
>77, d1[0.883] d2[0.885] g[0.075]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step
>78, d1[0.883] d2[0.885] g

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 57ms/step
>142, d1[0.892] d2[0.893] g[0.046]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 54ms/step
>143, d1[0.892] d2[0.893] g[0.045]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 59ms/step
>144, d1[0.892] d2[0.894] g[0.045]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 57ms/step
>145, d1[0.893] d2[0.894] g[0.045]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 58ms/step
>146, d1[0.893] d2[0.894] g[0.044]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 60ms/step
>147, d1[0.893] d2[0.894] g[0.044]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 56ms/step
>148, d1[0.893] d2[0.894] g[0.044]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 56ms/step
>149, d1[0.893] d2[0.895] g[0.044]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 56ms/step
>150, d1[0.894] d2[0.895] g[0.044]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 56ms/step
>151, d1[0.894] d2[0.895] g[0.043]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 59ms/step
>152, d1[0.894] d2[0.895] g[0.043]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 62ms/step
>153, d1[0.894] d2[0.896] g[0.043]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 58ms/step
>154, d1[0.895] d2[0.896] g[0.043]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 57ms/step
>155, d1[0.89

...

Images and model saved at C:\Users\net pc\Desktop\LIVER TUMOR\benign_generated
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 937ms/step
>1001, d1[2.116] d2[2.118] g[0.010]
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 525ms/step
>1002, d1[2.117] d2[2.119] g[0.010]
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 537ms/step
>1003, d1[2.119] d2[2.121] g[0.010]
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 678ms/step
>1004, d1[2.120] d2[2.122] g[0.010]
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 598ms/step
>1005, d1[2.121] d2[2.123] g[0.010]
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 570ms/step
>1006, d1[2.122] d2[2.124] g[0.010]
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 565ms/step
>1007, d1[2.124] d2[2.126] g[0.010]
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 611ms/step
>1008, d1[2.125] d2[2.127] g[0.010]
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 693ms/step
>1009, d1[2.126] d2[2.128] g[0.010]
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 662ms/step
>1010, d1[2.127] d2[2.129] g[0.009]
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 681ms/step
>1011, d1[2.129] d2[2.131] g[0.009]
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 797ms/step
>1012, d1[2.130] d2[2.132] g[0.009]
1/1 ━━━━━━━━━━━━━━━━━

1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 680ms/step
>1075, d1[2.203] d2[2.205] g[0.009]
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
>1076, d1[2.204] d2[2.206] g[0.009]
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 627ms/step
>1077, d1[2.206] d2[2.208] g[0.009]
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 578ms/step
>1078, d1[2.207] d2[2.209] g[0.009]
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 536ms/step
>1079, d1[2.208] d2[2.210] g[0.009]
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 615ms/step
>1080, d1[2.209] d2[2.211] g[0.009]
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 617ms/step
>1081, d1[2.210] d2[2.212] g[0.009]
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 791ms/step
>1082, d1[2.211] d2[2.213] g[0.009]
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 774ms/step
>1083, d1[2.212] d2[2.214] g[0.009]
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 663ms/step
>1084, d1[2.213] d2[2.215] g[0.009]
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 841ms/step
>1085, d1[2.214] d2[2.216] g[0.009]
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 741ms/step
>1086, d1[2.216] d2[2.217] g[0.009]
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 559ms/step
>1087, d1[2.217] d2[2.218] g[0.009]
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 

1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 718ms/step
>1150, d1[2.282] d2[2.283] g[0.009]
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 655ms/step
>1151, d1[2.282] d2[2.284] g[0.009]
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 645ms/step
>1152, d1[2.283] d2[2.285] g[0.009]
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 655ms/step
>1153, d1[2.284] d2[2.286] g[0.009]
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 687ms/step
>1154, d1[2.285] d2[2.287] g[0.009]
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 683ms/step
>1155, d1[2.286] d2[2.288] g[0.009]
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 794ms/step
>1156, d1[2.287] d2[2.289] g[0.009]
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
>1157, d1[2.288] d2[2.290] g[0.009]
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 804ms/step
>1158, d1[2.289] d2[2.291] g[0.009]
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 885ms/step
>1159, d1[2.290] d2[2.292] g[0.009]
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 632ms/step
>1160, d1[2.291] d2[2.293] g[0.009]
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 772ms/step
>1161, d1[2.292] d2[2.294] g[0.009]
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 662ms/step
>1162, d1[2.293] d2[2.295] g[0.009]
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 

1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 810ms/step
>1225, d1[2.351] d2[2.353] g[0.008]
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
>1226, d1[2.352] d2[2.354] g[0.008]
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 777ms/step
>1227, d1[2.353] d2[2.354] g[0.008]
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 827ms/step
>1228, d1[2.354] d2[2.355] g[0.008]
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 778ms/step
>1229, d1[2.355] d2[2.356] g[0.008]
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 680ms/step
>1230, d1[2.356] d2[2.357] g[0.008]
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 664ms/step
>1231, d1[2.356] d2[2.358] g[0.008]
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 694ms/step
>1232, d1[2.357] d2[2.359] g[0.008]
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 703ms/step
>1233, d1[2.358] d2[2.360] g[0.008]
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 765ms/step
>1234, d1[2.359] d2[2.361] g[0.008]
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 847ms/step
>1235, d1[2.360] d2[2.361] g[0.008]
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
>1236, d1[2.361] d2[2.362] g[0.008]
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 728ms/step
>1237, d1[2.362] d2[2.363] g[0.008]
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 721

1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 734ms/step
>1300, d1[2.414] d2[2.416] g[0.008]
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 797ms/step
>1301, d1[2.415] d2[2.417] g[0.008]
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 859ms/step
>1302, d1[2.416] d2[2.418] g[0.008]
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 823ms/step
>1303, d1[2.417] d2[2.418] g[0.008]
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
>1304, d1[2.418] d2[2.419] g[0.008]
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 762ms/step
>1305, d1[2.418] d2[2.420] g[0.008]
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 719ms/step
>1306, d1[2.419] d2[2.421] g[0.008]
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 734ms/step
>1307, d1[2.420] d2[2.421] g[0.008]
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 703ms/step
>1308, d1[2.421] d2[2.422] g[0.008]
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 703ms/step
>1309, d1[2.422] d2[2.423] g[0.008]
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 765ms/step
>1310, d1[2.422] d2[2.424] g[0.008]
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 750ms/step
>1311, d1[2.423] d2[2.425] g[0.008]
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 773ms/step
>1312, d1[2.424] d2[2.425] g[0.008]
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 